In [9]:
import pandas as pd
import numpy as np
from sklearn.ensemble import IsolationForest
import warnings

warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', None)

In [10]:
df = pd.read_csv(r'..\..\datasets\df_minmax_norm.csv', encoding='latin-1')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2065547 entries, 0 to 2065546
Data columns (total 7 columns):
 #   Column            Dtype  
---  ------            -----  
 0   nf_timestamp      float64
 1   emit_lat          float64
 2   emit_long         float64
 3   prod_ncm          float64
 4   prod_quant        float64
 5   prod_valor_unit   float64
 6   prod_valor_total  float64
dtypes: float64(7)
memory usage: 110.3 MB


In [11]:
model = IsolationForest(
    random_state=26,
    verbose=True,
    contamination=0.0001
)

model.fit(df)

anomaly_predictions = model.predict(df)
df['anomaly_score'] = model.decision_function(df)

df_outliers = df[anomaly_predictions == -1]
df_outliers.head(5)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    2.1s finished


,nf_timestamp,emit_lat,emit_long,prod_ncm,prod_quant,prod_valor_unit,prod_valor_total,anomaly_score
617058,0.986435,0.71257,0.997562,0.901850,1.500000e-07,0.000254,0.003044,-0.000644
668827,0.994641,0.71257,0.997562,0.901850,1.500000e-07,0.000254,0.003044,-0.005267
790673,0.973785,0.71257,0.997562,0.902751,1.250000e-08,0.002680,0.002680,-0.002192
794659,0.966331,0.71257,0.997562,0.902781,1.250000e-08,0.001535,0.001535,-0.002192
806356,0.968863,0.71257,0.997562,0.902751,1.250000e-08,0.003360,0.003360,-0.002192


In [12]:
index_array = df_outliers.index.values
index_array

array([ 617058,  668827,  790673,  794659,  806356,  806357, 1010022,
       1010355, 1028814, 1030869, 1040213, 1089316, 1089341, 1557852,
       1557854, 1557936, 1558290, 1558513, 1558514, 1624955, 1624959,
       1625601, 1625677, 1626246, 1626326, 1626964, 1627731, 1627751,
       1628482, 1629189, 1629222, 1629977, 1630065, 1630547, 1631567,
       1631614, 1631699, 1631760, 1631818, 1631872, 1631882, 1631887,
       1631888, 1632039, 1632111, 1632112, 1632114, 1632159, 1632275,
       1632276, 1632407, 1632463, 1632562, 1632620, 1632621, 1632761,
       1632821, 1632822, 1632876, 1632877, 1632878, 1632922, 1633053,
       1633054, 1633169, 1633231, 1633387, 1633390, 1633391, 1633417,
       1633583, 1633632, 1633762, 1633911, 1633913, 1633964, 1633965,
       1634139, 1634196, 1634197, 1634241, 1634300, 1634506, 1634602,
       1634603, 1634888, 1634981, 1634982, 1648381, 1872934, 1919377,
       1919384, 1919435, 1919439, 1919440, 1919512, 1919516, 1919517,
       1919610, 1988

In [13]:
df_outliers['idx'] = df_outliers.index
df_outliers = df_outliers.sort_values(by='anomaly_score')
df_outliers.to_csv('output\outliers.csv', index=False)
df_outliers

,nf_timestamp,emit_lat,emit_long,prod_ncm,prod_quant,prod_valor_unit,prod_valor_total,anomaly_score,idx
2065189,0.995813,0.326848,0.318027,0.960910,6.387125e-04,4.729287e-08,0.002417,-0.021934,2065189
2064097,0.298928,0.053435,0.436355,0.000000,2.500000e-08,1.791397e-03,0.003583,-0.019398,2064097
2065186,0.995813,0.326848,0.318027,0.960820,3.703625e-04,6.449028e-08,0.001911,-0.019289,2065186
2065190,0.995813,0.326848,0.318027,0.960910,2.963675e-03,3.439481e-09,0.000815,-0.018761,2065190
1988875,0.038332,0.237667,0.582565,0.902751,1.250000e-08,5.991419e-03,0.005991,-0.015075,1988875
...,...,...,...,...,...,...,...,...,...
1633390,0.911731,0.619720,0.366796,0.490110,1.491587e-03,6.433263e-07,0.076766,-0.000268,1633390
1633391,0.911731,0.619720,0.366796,0.490110,1.491587e-03,6.823071e-07,0.081418,-0.000268,1633391
2059327,0.991056,0.261073,0.568905,0.901840,9.000000e-04,4.299352e-09,0.000306,-0.000098,2059327
2059038,0.996200,0.258691,0.718174,0.854371,2.300000e-06,1.604518e-04,0.029523,-0.000085,2059038


In [14]:
index_df = pd.DataFrame(index_array, columns=['idx'])
index_df.to_csv('output\outlier_indexes.csv', index=True)

In [16]:
dft = pd.read_csv(r'output\outliers.csv', encoding='latin-1')
dft.set_index('idx', inplace=True)
dft = dft.sort_values(by='anomaly_score', ascending=True)
dft.head(5)

,nf_timestamp,emit_lat,emit_long,prod_ncm,prod_quant,prod_valor_unit,prod_valor_total,anomaly_score
idx,,,,,,,,
2065189,0.995813,0.326848,0.318027,0.960910,6.387125e-04,4.729287e-08,0.002417,-0.021934
2064097,0.298928,0.053435,0.436355,0.000000,2.500000e-08,1.791397e-03,0.003583,-0.019398
2065186,0.995813,0.326848,0.318027,0.960820,3.703625e-04,6.449028e-08,0.001911,-0.019289
2065190,0.995813,0.326848,0.318027,0.960910,2.963675e-03,3.439481e-09,0.000815,-0.018761
1988875,0.038332,0.237667,0.582565,0.902751,1.250000e-08,5.991419e-03,0.005991,-0.015075
